In [1]:
import get_schedule
import pandas as pd


In [2]:
# login_code: str,
#         start_date: str,
#         end_date: str,
#         names: list,
schedule, parsed_dates, raw_schedule = get_schedule.Schedule().run(
        login_code="chla",
        start_date="2023-08-02",
        end_date="2023-08-07",
        names=['Benedetto, Lauren', "Freeburg, Taylor"]
    )

reading from cache


In [3]:
schedule

,name,team,start_time,end_time
61,"Freeburg, Taylor",Team 5&6 Night Sr,2023-08-02 17:00:00,2023-08-02 07:00:00
132,"Benedetto, Lauren",Team 1&2 Night Sr,2023-08-03 17:00:00,2023-08-03 06:00:00
134,"Freeburg, Taylor",Team 5&6 Night Sr,2023-08-03 17:00:00,2023-08-03 07:00:00
202,"Benedetto, Lauren",Team 1&2 Night Sr,2023-08-04 17:00:00,2023-08-04 06:00:00
204,"Freeburg, Taylor",Team 5&6 Night Sr,2023-08-04 17:00:00,2023-08-04 07:00:00
254,"Benedetto, Lauren",Team 1&2 Night Sr,2023-08-05 17:00:00,2023-08-05 12:00:00
256,"Freeburg, Taylor",Team 5&6 Night Sr,2023-08-05 17:00:00,2023-08-05 12:00:00


In [6]:
scope = pd.DataFrame(
    pd.date_range(
        start=parsed_dates["start_date"],
        periods=int(parsed_dates['days']) * 24,
        freq="1H",
    ),
    columns=["date"],
)

In [7]:
scope

,date
0,2023-07-01 00:00:00
1,2023-07-01 01:00:00
2,2023-07-01 02:00:00
3,2023-07-01 03:00:00
4,2023-07-01 04:00:00
...,...
91,2023-07-04 19:00:00
92,2023-07-04 20:00:00
93,2023-07-04 21:00:00
94,2023-07-04 22:00:00


In [3]:
start_year=2023
start_month=7
start_day=1
days=30

In [ ]:
working_hours

In [ ]:
working_hours.explode("hour").reset_index()

In [5]:
schedule

,timestamp,date,hour,"Benedetto, Lauren","Freeburg, Taylor",free_time
0,2023-07-01 00:00:00,2023-07-01,0,0.0,0.0,True
1,2023-07-01 01:00:00,2023-07-01,1,0.0,0.0,True
2,2023-07-01 02:00:00,2023-07-01,2,0.0,0.0,True
3,2023-07-01 03:00:00,2023-07-01,3,0.0,0.0,True
4,2023-07-01 04:00:00,2023-07-01,4,0.0,0.0,True
...,...,...,...,...,...,...
91,2023-07-04 19:00:00,2023-07-04,19,1.0,0.0,False
92,2023-07-04 20:00:00,2023-07-04,20,1.0,0.0,False
93,2023-07-04 21:00:00,2023-07-04,21,1.0,0.0,False
94,2023-07-04 22:00:00,2023-07-04,22,1.0,0.0,False
